# Script for realtime predictions with tensorflow an a self made dataset

## Import all nessesary dependencies with time check

In [ ]:
# Import time for performance measurements
import time
import numpy as np

# Start time measuring
start_time = time.time()

# Import picamera
import picamera
import picamera.array
print("--- %s seconds for picamera imports ---" % (time.time() - start_time))

#Import tflearn libraries
from tflearn import DNN
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
print("--- %s seconds for tflearn imports ---" % (time.time() - start_time))


## Create the convolutional neural network with realtime image preprocessing

In [ ]:
# Start time measuring
start_time = time.time()

# Real-time data augmentation (This is only used while training the DNN)
img_aug = ImageAugmentation()
img_aug.add_random_blur(0.25)
img_aug.add_random_rotation(max_angle=10.0)

# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()
print("--- %s seconds for image preprocessing functions ---" % (time.time() - start_time))

# Convolutional network building
network = input_data(shape=[None, 120, 160, 3],
                     data_augmentation=img_aug,
                    data_preprocessing=img_prep)
network = conv_2d(network, 64, 5, activation='relu')
network = max_pool_2d(network, 4)
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 128, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 256, activation='relu')
network = dropout(network, 1)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

model = DNN(network, tensorboard_verbose=0)
print("--- %s seconds for creating the neural network ---" % (time.time() - start_time))

# Load the model
model.load('models/final_model_aug_prep_4')
print("--- %s seconds for importing the model ---" % (time.time() - start_time))

## Get camera images and transform them to numpy arrays for live feeding into our cnn

In [ ]:
from matplotlib.pyplot import imshow
np.set_printoptions(precision=4)
%matplotlib inline

In [ ]:
# Capture images for prediction
with picamera.PiCamera() as camera:
    camera.resolution = (160, 120)
    time.sleep(1)
    with picamera.array.PiRGBArray(camera) as output:
        for i in range(60):
            
            camera.capture(output, 'rgb')
            imshow(output.array)
            
            # Copy and reshape
            X = np.array(output.array)
            X = X.astype(float)
            X = X.reshape((1, 120, 160, 3))
            
            # Make prediction
            prediction_time = time.time()
            prediction = np.round(model.predict(X), decimals=2)
            print("                                                                ", end="\r")
            print(prediction, end="\r")
            
            #print("--- %s seconds for the prediction ---" % (time.time() - prediction_time))
            output.truncate(0)
        